In [68]:
import numpy as np 
##Loading the data 
raw_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

raw_inputs = raw_data[:, 1:-1]
raw_targets = raw_data[:, -1]

In [61]:
## I will use mulitple techniques for preprocessing in order to check which yields the best results
## I will vary shuffling and scaling the features and balancing 
## Manual Balancing, Shuffling Splitting 

In [ ]:
## Manual Balancing
import numpy as np 
 
num_ones = int(np.sum(raw_targets))
zeros = 0
to_remove = []


for i in range(raw_targets.shape[0]):
    if raw_targets[i] == 0:
        zeros += 1
        if zeros > num_ones:
            to_remove.append(i)


manual_bal_ins = np.delete(raw_inputs, to_remove, axis=0)
manual_bal_outs = np.delete(raw_targets, to_remove, axis=0)

In [111]:
##Scale
manual_bal_ins = preprocessing.scale(manual_bal_ins)

In [108]:
## Manual Shuffling 
shuffled = np.arange(manual_bal_ins.shape[0])
np.random.shuffle(shuffled)

man_Bal_Shuffle_ins = manual_bal_ins[shuffled]
man_Bal_Shuffle_outs = manual_bal_outs[shuffled]

In [110]:
## Manual Splitting 
sampled = man_Bal_Shuffle_ins.shape[0]


# Naturally, the numbers are integers.
train_count = int(0.8 * sampled)
validation_count = int(0.1 * sampled)
test_count = sampled - train_count - validation_count


man_ins_train = man_Bal_Shuffle_ins[:train_count]
man_outs_train = man_Bal_Shuffle_outs[:train_count]

man_ins_validation = man_Bal_Shuffle_ins[train_count:train_count+validation_count]
man_outs_validation = man_Bal_Shuffle_outs[train_count:train_count+validation_count]

man_ins_test = man_Bal_Shuffle_ins[train_count+validation_count:]
man_outs_test = man_Bal_Shuffle_outs[train_count+validation_count:]


In [ ]:
## Sets
## Status: Balanced, Scaled, Shuffled, Split 
## Usage: Manual
"""
man_ins_test, 
man_ins_train, 
man_ins_validation
"""

In [88]:
## Balancing through Imbalanced Learn 
## Naive Under Sampling 
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
Under_sampler = RandomUnderSampler(random_state=69, replacement=False)
x_underSampled, y_underSampled = Under_sampler.fit_resample(raw_inputs, raw_targets)

In [93]:
##Random Over sampling using Syntethics 
from imblearn.over_sampling import SMOTE 
Synth_Over_Sampler = SMOTE()
x_OverSampled, y_OverSampled = Synth_Over_Sampler.fit_resample(raw_inputs, raw_targets)

In [94]:
## Under sampling using Distance Metrics 
from imblearn.under_sampling import TomekLinks
Tomekker = TomekLinks()
x_UnderSampled_Dis, y_UnderSampled_Dis = Tomekker.fit_resample(raw_inputs, raw_targets)

In [101]:
## Scaling the data 
from sklearn import preprocessing
x_Scaled_OS = preprocessing.scale(x_OverSampled)
x_Scaled_US = preprocessing.scale(x_underSampled)
x_Scaled_Dis = preprocessing.scale(x_UnderSampled_Dis)

In [104]:
## shuffling and splitting
from sklearn.model_selection import train_test_split

x_Scaled_Dis_train, x_Scaled_Dis_test, y_UnderSampled_Dis_train, y_UnderSampled_Dis_test = train_test_split(
    x_Scaled_Dis, y_UnderSampled_Dis
)
x_Scaled_OS_train, x_Scaled_OS_test, y_UnderSampled_OS_train, y_UnderSampled_OS_test = train_test_split(
    x_Scaled_OS, y_OverSampled
)
x_Scaled_US_train, x_Scaled_US_test, y_UnderSampled_US_train, y_UnderSampled_US_test = train_test_split(
    x_Scaled_US, y_underSampled
)

In [ ]:
## Sets till now
## Status: Balanced, Scaled, Split, Randomized.
## Used: Imbalanced Learn for shuffling 
"""
x,y_Scaled_Dis - train, test -> Under Sampled using Distance metric
x,y_Scaled_OS - train, test -> Over Sampled using Random
x,y_Scaled_US - train, test -> Under Sampled using Random
"""
## 
"""
man_ins_test, 
man_ins_train, 
man_ins_validation
"""
## Status: Balanced, Scaled, Split, Randomized
## Used: Manual

In [112]:
## Train a simple model and see the difference between manual and module methods
import tensorflow as tf 

In [131]:
## Simple DNN with optional early stopping for manual set 
input_size = 10 
output_size = 2

layer_size = 50
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(layer_size, 'relu'),
    tf.keras.layers.Dense(layer_size, 'relu'),
    tf.keras.layers.Dense(output_size, 'sigmoid')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [150]:
##Print the results of the evaluation of each set 
def fitting(x, x_test, y, y_test, _callbacks, val_data):
   
    model.fit(
        x=x,
        y=y, 
        batch_size=50,
        epochs=50,
        callbacks=_callbacks,
        validation_data=val_data,
        verbose=2
    )

    results = model.evaluate(x=x_test, y=y_test)
    


    print('loss, acc:', results)

In [152]:
fitting(man_ins_train, 
        man_ins_test, 
        man_outs_train, 
        man_outs_test, early_stopping, [man_ins_validation, man_outs_validation])

Epoch 1/50
72/72 - 0s - loss: 0.3711 - accuracy: 0.7893 - val_loss: 0.3412 - val_accuracy: 0.7852 - 174ms/epoch - 2ms/step
Epoch 2/50
72/72 - 0s - loss: 0.3541 - accuracy: 0.7910 - val_loss: 0.4140 - val_accuracy: 0.7696 - 143ms/epoch - 2ms/step
Epoch 3/50
72/72 - 0s - loss: 0.3551 - accuracy: 0.7930 - val_loss: 0.4360 - val_accuracy: 0.7763 - 138ms/epoch - 2ms/step
14/14 [==============================] - 0s 653us/step - loss: 0.3579 - accuracy: 0.8214
loss, acc: [0.35787925124168396, 0.8214285969734192]


In [156]:
fitting(x_Scaled_Dis_train,
        x_Scaled_Dis_test,
        y_UnderSampled_Dis_train,
        y_UnderSampled_Dis_test, None, None)
fitting(x_Scaled_US_train,
        x_Scaled_US_test,
        y_UnderSampled_US_train,
        y_UnderSampled_US_test, None, None)
fitting(x_Scaled_OS_train,
        x_Scaled_OS_test,
        y_UnderSampled_OS_train,
        y_UnderSampled_OS_test, None, None)

Epoch 1/50
208/208 - 0s - loss: 0.2177 - accuracy: 0.9123 - 302ms/epoch - 1ms/step
Epoch 2/50
208/208 - 0s - loss: 0.2189 - accuracy: 0.9110 - 317ms/epoch - 2ms/step
Epoch 3/50
208/208 - 0s - loss: 0.2185 - accuracy: 0.9120 - 300ms/epoch - 1ms/step
Epoch 4/50
208/208 - 0s - loss: 0.2187 - accuracy: 0.9112 - 339ms/epoch - 2ms/step
Epoch 5/50
208/208 - 0s - loss: 0.2173 - accuracy: 0.9123 - 330ms/epoch - 2ms/step
Epoch 6/50
208/208 - 0s - loss: 0.2178 - accuracy: 0.9124 - 328ms/epoch - 2ms/step
Epoch 7/50
208/208 - 0s - loss: 0.2174 - accuracy: 0.9123 - 334ms/epoch - 2ms/step
Epoch 8/50
208/208 - 0s - loss: 0.2183 - accuracy: 0.9117 - 334ms/epoch - 2ms/step
Epoch 9/50
208/208 - 0s - loss: 0.2183 - accuracy: 0.9112 - 296ms/epoch - 1ms/step
Epoch 10/50
208/208 - 0s - loss: 0.2171 - accuracy: 0.9106 - 300ms/epoch - 1ms/step
Epoch 11/50
208/208 - 0s - loss: 0.2169 - accuracy: 0.9120 - 317ms/epoch - 2ms/step
Epoch 12/50
208/208 - 0s - loss: 0.2173 - accuracy: 0.9112 - 304ms/epoch - 1ms/step
E

In [179]:
## Wrap the best performing model for deployment using tabpy
from keras_pickle_wrapper import KerasPickleWrapper
import pickle
import keras 
best_fit = KerasPickleWrapper(model)

In [180]:
## Setup tapby for Tableau connection
from tabpy.tabpy_tools.client import Client
from tabpy import tabpy_tools

client = Client('http://localhost:9004/')
client.set_credentials('Yourname', 'PassHere')

In [181]:
## Write a passable function to wrap the model in 
## This code has been adopted from a very handy tabpy tutorial!
# https://medium.com/geekculture/deep-learning-in-tableau-using-a-keras-neural-network-and-python-tabpy-8f8716c35488
def PredictedConversion(_arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8, _arg9, _arg10):
    import pandas as pd
    import joblib
    
    row = {'Book Length Overall': _arg1,
           'Book Length Average': _arg2,
           'Price Overall': _arg3,
           'Price Average': _arg4,
           
           'Review': _arg5,
           'Review10/10': _arg6,
           'Minutes Listened': _arg7,
           'Completion': _arg8,
           'Support Requests': _arg9,
           
           'Last Visited Minus Purchase Date': _arg10,

           }

    
    X = pd.DataFrame(data = row, index=[0])
    return str(best_fit().predict(X)[0][0])  